[View in Colaboratory](https://colab.research.google.com/github/thayumaanavan/TensorFlow/blob/master/Intro_to_TensorFlow.ipynb)

In [0]:
from __future__ import absolute_import,division,print_function

import numpy as np
import tensorflow as tf